In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import logging
import re
logging.basicConfig(filename='pack.log', level=logging.INFO)
from scipy.optimize import minimize



In [2]:
# упаковки:
three_layer = np.array([
    [0, 0, 0],
    [0, 0.5, 0.5],
    [0.5, 0, 0.5],
    [0.5, 0.5, 0]
])

oct_void = np.array([
    [0.5, 0.5, 0.5],
    [0.5, 0, 0],
    [0, 0.5, 0],
    [0, 0, 0.5]
])

tetr_void = np.array([
    [0.25, 0.25, 0.25],
    [0.25, 0.75, 0.75],
    [0.75, 0.25, 0.75],
    [0.75, 0.75, 0.25],
    [0.75, 0.75, 0.75],
    [0.75, 0.25, 0.25],
    [0.25, 0.75, 0.25],
    [0.25, 0.25, 0.75]
])

In [3]:
def get_system_from_cif(cif_file):
    #поиск блока с общей пст
    block_pattern = re.compile(r'loop_\s*_symmetry_equiv_pos_site_id\s*_symmetry_equiv_pos_as_xyz\s*(.*?)\n(?=loop_|$)', re.DOTALL)
    block_match = block_pattern.search(cif_file)
    
    if block_match:
        # Извлекаю найденный блок
        block_content = block_match.group(1)
        
        # Разделяю блок на строки
        lines = block_content.strip().split('\n')
        
        # извлекаю координаты
        coordinates = []
        for line in lines:
            # удаляю лишнее
            cleaned_line = re.sub(r'\d+\s', '', line).replace("'", "")
            # Добавляю координаты в список
            if cleaned_line.strip(): 
                coordinates.append(cleaned_line.strip())
        
        return coordinates
    else:
        return None

In [4]:

#Размножение координаты по ПСТ
def expand_positions(system_points, pos):
    x, y, z = pos
    new_coords = []
    for pos in system_points:
        new_pos = pos.replace('x', str(x)).replace('y', str(y)).replace('z', str(z))
        if '--' in new_pos: 
            new_pos = new_pos.replace('--','-') 
        parts = new_pos.split(',')
        new_part = []
        for part in parts:
            if '/' in part:
                denom = 2.0
                num = 1.0
                if '1/4' in part:
                    denom = 4.0
                if '1/3' in part:
                    num = 3.0
                if '+' in part:
                    sub_parts = part.split('+')
                    value = (num / denom) + float(sub_parts[1])
                elif '-' in part:
                    sub_parts = part.split('-')
                    value = (num / denom) - float(sub_parts[1])
            else:
                value = float(part)
            new_part.append(value)
        new_coords.append(new_part)
    return new_coords

In [5]:
def unique_positions(arr):
    # unique_rows = np.unique(arr, axis=0)
    # return unique_rows
    unique_list = []
    for sublist in arr:
        if sublist not in unique_list:
            unique_list.append(sublist)
    return np.array(unique_list)

In [13]:
# Функция для минимизации
def objective(delta, O, S):
    return np.sum(np.linalg.norm(O - S + delta, axis=1)**2)

# Функция для минимизации с векторами O и S в качестве аргументов
def minimize_delta(O, S):
    # Начальное предположение для вектора delta
    initial_guess = np.zeros_like(O[0])
    # Минимизация функции
    result = minimize(objective, initial_guess, args=(O, S))
    # Полученный вектор delta
    delta = result.x
    return delta

def check_pack(orbit, pack ,d):
    one = np.array_equal(orbit + d, pack)
    two = np.array_equal(orbit - d, pack)
    return one or two

In [14]:
data = pd.read_csv('cubic_groups.csv')

#подключение к бд
conn = mysql.connector.connect(host='localhost', user='user', password='12345', database='NNCDB')
cursor = conn.cursor()
cnt = 0
for i in range(len(data)):
    item = data.iloc[i]
    id = item['CCDC_ID']
    #запрашиваю cif-файл, чтобы взять общую ПСТ
    query = 'SELECT CIF_FILE FROM CIFS WHERE ID = %s'
    cursor.execute(query, (int(id),))
    resp_res = cursor.fetchall()   

    cif = resp_res[0][0]

    #получаю общую ПСТ
    psys = get_system_from_cif(cif)

    center_of_mass = [float(item['C_x']), float(item['C_y']), float(item['C_z'])]

    #размножаю ЦМ по общей ПСТ
    orbit = expand_positions(psys, center_of_mass)

    #удаляю совпадения
    uniq_orbit = unique_positions(orbit)
    if (len(uniq_orbit) == 4):
        logging.info(f"[+] Молекула {item['REFCODE']}({item['Formula']})\n")
        logging.info(uniq_orbit)
        delta1 = minimize_delta(uniq_orbit, three_layer)
        delta2 = minimize_delta(uniq_orbit, oct_void)
        logging.info(f"[+] d={delta1}")
        logging.info(f"[+] d={delta2}")
        if check_pack(uniq_orbit, three_layer, delta1):
            logging.info(f"[+] 3-х слойная")
        elif check_pack(uniq_orbit, three_layer, delta2):
            logging.info(f"[+] окт. пустоты")
        else:
            logging.info(f"[+] Упаковка не найдена")
    elif (len(uniq_orbit) == 8):
        logging.info(f"[+] Молекула {item['REFCODE']}({item['Formula']})\n")
        logging.info(uniq_orbit)
        delta = minimize_delta(uniq_orbit, tetr_void)
        logging.info(f"[+] d={delta}")
        if check_pack(uniq_orbit, tetr_void, delta):
            logging.info(f"[+] тетр. пустоты")
        else:
            logging.info(f"[+] Упаковка не найдена")


cursor.close()
conn.close()

In [8]:
# for checking:

# item = data.iloc[3]
# id = item['CCDC_ID']
# print(id)
# #запрашиваю cif-файл, чтобы взять общую ПСТ
# query = 'SELECT CIF_FILE FROM CIFS WHERE ID = %s'
# cursor.execute(query, (int(id),))
# resp_res = cursor.fetchall()   

# cif = resp_res[0][0]

# #получаю общую ПСТ
# psys = get_system_from_cif(cif)
# center_of_mass = [float(item['C_x']), float(item['C_y']), float(item['C_z'])]
# print(center_of_mass)
# #размножаю ЦМ по общей ПСТ
# orbit = expand_positions(psys, center_of_mass)

# #удаляю совпадения
# uniq_orbit = unique_positions(orbit)
# print(uniq_orbit)